In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
from PIL import Image, ImageTk
import tensorflow as tf
import cv2
import numpy as np
import os

class DentalClassifierGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Dental X-ray Classification with Recommendations")
        self.root.geometry("1200x800")
        
        # Load model
        try:
            self.model = tf.keras.models.load_model('high_accuracy_dental_model_1.h5')
            self.class_names = ['Cavity', 'Fillings', 'Impacted Tooth', 'Implant']
            
            # Recommendation database
            self.recommendations = {
                'Cavity': {
                    'Emergency': [
                        "Immediate dental consultation required",
                        "Temporary pain management with over-the-counter medication",
                        "Avoid very hot or cold foods",
                        "Use temporary dental filling if available"
                    ],
                    'Treatment': [
                        "Professional dental filling procedure",
                        "Root canal may be necessary if decay is severe",
                        "Regular fluoride treatment",
                        "Dental sealants for prevention"
                    ],
                    'Prevention': [
                        "Brush teeth twice daily with fluoride toothpaste",
                        "Daily flossing",
                        "Reduce sugar intake",
                        "Regular dental check-ups every 6 months"
                    ]
                },
                'Fillings': {
                    'Maintenance': [
                        "Regular dental check-ups to monitor filling condition",
                        "Avoid biting hard objects",
                        "Use fluoride toothpaste",
                        "Report any sensitivity or discomfort"
                    ],
                    'Prevention': [
                        "Maintain good oral hygiene",
                        "Use soft-bristled toothbrush",
                        "Consider night guard if you grind teeth",
                        "Regular professional cleaning"
                    ]
                },
                'Impacted Tooth': {
                    'Emergency': [
                        "Consult oral surgeon for evaluation",
                        "Pain management as recommended by dentist",
                        "Apply ice pack for swelling",
                        "Soft food diet recommended"
                    ],
                    'Treatment': [
                        "Surgical extraction may be necessary",
                        "Orthodontic treatment might be recommended",
                        "Regular monitoring if surgery is delayed",
                        "X-ray monitoring of tooth position"
                    ],
                    'Prevention': [
                        "Early orthodontic evaluation",
                        "Regular dental check-ups",
                        "Monitor wisdom teeth development",
                        "Follow post-treatment care instructions"
                    ]
                },
                'Implant': {
                    'Maintenance': [
                        "Regular professional cleaning",
                        "Use special implant-specific cleaning tools",
                        "Avoid smoking",
                        "Monitor for any looseness or discomfort"
                    ],
                    'Care': [
                        "Gentle brushing around implant area",
                        "Use recommended antimicrobial mouthwash",
                        "Regular check-ups with dentist",
                        "Report any unusual symptoms immediately"
                    ],
                    'Long-term': [
                        "Maintain excellent oral hygiene",
                        "Avoid hard or sticky foods",
                        "Regular x-ray monitoring",
                        "Professional evaluation every 6 months"
                    ]
                }
            }
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load model: {str(e)}")
            self.root.destroy()
            return
        
        self.create_widgets()
        
    def create_widgets(self):
        # Main frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Control frame (left side)
        control_frame = ttk.Frame(main_frame)
        control_frame.grid(row=0, column=0, sticky=(tk.N, tk.S))
        
        # Upload button
        ttk.Button(control_frame, text="Upload Image", command=self.upload_image).grid(row=0, column=0, pady=10)
        
        # Results frame
        self.results_frame = ttk.LabelFrame(control_frame, text="Classification Results", padding="10")
        self.results_frame.grid(row=1, column=0, pady=10)
        
        # Prediction label
        self.prediction_label = ttk.Label(self.results_frame, text="")
        self.prediction_label.grid(row=0, column=0, columnspan=4, pady=5)
        
        # Confidence bars
        self.confidence_bars = []
        self.confidence_labels = []
        for i in range(4):
            label = ttk.Label(self.results_frame, text=self.class_names[i])
            label.grid(row=i+1, column=0, padx=5, pady=2)
            
            progress = ttk.Progressbar(self.results_frame, length=200, mode='determinate')
            progress.grid(row=i+1, column=1, padx=5, pady=2)
            
            value_label = ttk.Label(self.results_frame, text="0%")
            value_label.grid(row=i+1, column=2, padx=5, pady=2)
            
            self.confidence_bars.append(progress)
            self.confidence_labels.append(value_label)
        
        # Image display (middle)
        self.image_frame = ttk.LabelFrame(main_frame, text="X-ray Image", padding="10")
        self.image_frame.grid(row=0, column=1, padx=10)
        self.image_label = ttk.Label(self.image_frame)
        self.image_label.grid(row=0, column=0)
        
        # Recommendations frame (right side)
        self.recommendations_frame = ttk.LabelFrame(main_frame, text="Recommendations", padding="10")
        self.recommendations_frame.grid(row=0, column=2, padx=10, sticky=(tk.N, tk.S))
        
        # Recommendations text area
        self.recommendations_text = scrolledtext.ScrolledText(
            self.recommendations_frame, 
            width=40, 
            height=30, 
            wrap=tk.WORD
        )
        self.recommendations_text.grid(row=0, column=0, padx=5, pady=5)
        
        # Store original image
        self.original_image = None
        self.current_photo = None
    
    def get_recommendations(self, predictions):
        recommendations_text = "DENTAL RECOMMENDATIONS\n"
        recommendations_text += "=" * 40 + "\n\n"
        
        # Sort predictions by confidence
        pred_with_class = list(zip(predictions[0], self.class_names))
        pred_with_class.sort(reverse=True)  # Sort by confidence
        
        threshold = 20  # Only show recommendations for conditions with >20% confidence
        
        for confidence, class_name in pred_with_class:
            confidence_pct = float(confidence) * 100
            if confidence_pct > threshold:
                recommendations_text += f"\n{class_name.upper()} ({confidence_pct:.1f}%)\n"
                recommendations_text += "-" * 40 + "\n"
                
                # Add recommendations for each category
                for category, items in self.recommendations[class_name].items():
                    recommendations_text += f"\n{category}:\n"
                    for item in items:
                        recommendations_text += f"• {item}\n"
                recommendations_text += "\n"
        
        if all(float(conf) * 100 <= threshold for conf, _ in pred_with_class):
            recommendations_text += "\nNo significant dental conditions detected.\n"
            recommendations_text += "Recommended: Regular dental check-up and maintenance.\n"
        
        return recommendations_text
    
    def upload_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        
        if file_path:
            try:
                # Load and process image
                self.process_image(file_path)
            except Exception as e:
                messagebox.showerror("Error", f"Error processing image: {str(e)}")
    
    def preprocess_image(self, image):
        # Resize to model input size
        resized = cv2.resize(image, (224, 224))
        # Normalize
        processed = resized / 255.0
        # Add batch dimension
        processed = np.expand_dims(processed, axis=0)
        return processed
    
    def process_image(self, image_path):
        # Read and convert image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Preprocess image for model
        processed_image = self.preprocess_image(image)
        
        # Get prediction
        prediction = self.model.predict(processed_image)
        
        # Get predicted class and confidence
        predicted_class_idx = np.argmax(prediction[0])
        predicted_class = self.class_names[predicted_class_idx]
        confidence = float(prediction[0][predicted_class_idx]) * 100
        
        # Update prediction label
        self.prediction_label.configure(
            text=f"Primary Detection: {predicted_class} ({confidence:.2f}%)"
        )
        
        # Display original image
        display_size = (500, 500)
        display_image = cv2.resize(image, display_size)
        self.current_photo = ImageTk.PhotoImage(Image.fromarray(display_image))
        self.image_label.configure(image=self.current_photo)
        
        # Update confidence bars
        for i, (prob, bar, label) in enumerate(zip(prediction[0], 
                                                 self.confidence_bars, 
                                                 self.confidence_labels)):
            confidence = float(prob) * 100
            bar['value'] = confidence
            label.configure(text=f"{confidence:.1f}%")
        
        # Generate and display recommendations
        recommendations = self.get_recommendations(prediction)
        self.recommendations_text.delete(1.0, tk.END)
        self.recommendations_text.insert(tk.END, recommendations)

def main():
    root = tk.Tk()
    app = DentalClassifierGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
